# B09705039_劉惟恩_HW05

## Q1

In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
from PIL import Image
import os
import glob
import random

# count amount of all labels and folders
labels_out = ['blazer', 'cardigan', 'coat', 'jacket']
folder = ['train', 'valid', 'test']
all_count = dict()

for j in range(3):
    temp_count = dict()
    for i in range(4):
        basepath = os.path.join("/tmp/dataset-ntu-photo/photos", folder[j], labels_out[i], "*.jpg")
        cand_fn = glob.glob(basepath)
        count = 0
        for afn in cand_fn:
            count += 1
        temp_count[labels_out[i]] = count
    all_count[folder[j]] = temp_count

In [2]:
# sum of all folders
for j in range(3):
    print(folder[j] + " total count:", sum(all_count[folder[j]].values()))

train total count: 1041
valid total count: 105
test total count: 146


In [3]:
# proportion of each label
for j in range(3):
    print(folder[j] + ":")
    total = sum(all_count[folder[j]].values())
    for i in range(4):
        print(labels_out[i] + " count:", all_count[folder[j]][labels_out[i]])
        print(labels_out[i] + " porportion:", round(all_count[folder[j]][labels_out[i]] * 100 / total, 2), "%")
    print()

train:
blazer count: 97
blazer porportion: 9.32 %
cardigan count: 237
cardigan porportion: 22.77 %
coat count: 296
coat porportion: 28.43 %
jacket count: 411
jacket porportion: 39.48 %

valid:
blazer count: 7
blazer porportion: 6.67 %
cardigan count: 36
cardigan porportion: 34.29 %
coat count: 27
coat porportion: 25.71 %
jacket count: 35
jacket porportion: 33.33 %

test:
blazer count: 9
blazer porportion: 6.16 %
cardigan count: 42
cardigan porportion: 28.77 %
coat count: 43
coat porportion: 29.45 %
jacket count: 52
jacket porportion: 35.62 %



Since blazer has the least data we expect that the accuracy of blazer will be lower than all the other labels. Cardigan and coat will have a similar accuracy while coat may be slightly higher due to it has larger training data. Finally, jacket is expected to have the highest accuracy among all the other labels since it has the largest training data.

## Q2

Import and setup

In [4]:
_exp_name = "q2"

In [5]:
# Import necessary packages.
import numpy as np
import pandas as pd
import torch
import os
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
# "ConcatDataset" and "Subset" are possibly useful when doing semi-supervised learning.
from torch.utils.data import ConcatDataset, DataLoader, Subset, Dataset
from torchvision.datasets import DatasetFolder, VisionDataset
import torchvision

# This is for the progress bar.
from tqdm.auto import tqdm
import random

In [6]:
myseed = 6666  # set a random seed for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(myseed)
torch.manual_seed(myseed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(myseed)

Transform picture

In [7]:
# Normally, We don't need augmentations in testing and validation.
# All we need here is to resize the PIL image and transform it into Tensor.
test_tfm = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# However, it is also possible to use augmentation in the testing phase.
# You may use train_tfm to produce a variety of images and then test using ensemble methods
train_tfm = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

Dataset

In [8]:
class Dataset(Dataset):

    def __init__(self,path,tfm=test_tfm,files = None):
        super(Dataset).__init__()
        self.path = path
        self.files = sorted([os.path.join(path, labels_out[i], j) for i in range(4) for j in os.listdir(os.path.join(path, labels_out[i])) if not j.startswith(".")])
        if files != None:
            self.files = files
        print(f"One {path} sample",self.files[0])
        self.transform = tfm
  
    def __len__(self):
        return len(self.files)
  
    def __getitem__(self,idx):
        fname = self.files[idx]
        im = Image.open(fname)
        im = self.transform(im)
        #im = self.data[idx]
        label = -2
        lname = fname.split("/")[-2]
        if lname == labels_out[0]:
            label = 0
        elif lname == labels_out[1]:
            label = 1
        elif lname == labels_out[2]:
            label = 2
        elif lname == labels_out[3]:
            label = 3
        else:
            label = -1
        return im,label

Training model

In [9]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        model = torchvision.models.resnet50(pretrained=True)
        model.fc = nn.Linear(2048, 4)
        self.model = model

    def forward(self, x):
        out = self.model(x)
        out = out.view(out.size()[0], -1)
        return out

Execute

In [10]:
batch_size = 32
_dataset_dir = "/tmp/dataset-ntu-photo/photos"
# Construct datasets.
# The argument "loader" tells how torchvision reads the data.
train_set = Dataset(os.path.join(_dataset_dir,"train"), tfm=train_tfm)
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)
valid_set = Dataset(os.path.join(_dataset_dir,"valid"), tfm=test_tfm)
valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)

One /tmp/dataset-ntu-photo/photos/train sample /tmp/dataset-ntu-photo/photos/train/blazer/20170324101342210_500.jpg
One /tmp/dataset-ntu-photo/photos/valid sample /tmp/dataset-ntu-photo/photos/valid/blazer/20170324142214231_500.jpg


In [11]:
# "cuda" only when GPUs are available.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

# The number of training epochs and patience.
n_epochs = 200
patience = 20 # If no improvement in 'patience' epochs, early stop

# Initialize a model, and put it on the device specified.
model = Classifier().to(device)

# For the classification task, we use cross-entropy as the measurement of performance.
criterion = nn.CrossEntropyLoss()

# Initialize optimizer, you may fine-tune some hyperparameters such as learning rate on your own.
optimizer = torch.optim.Adam(model.parameters(), lr=0.0003, weight_decay=1e-5) 

# Initialize trackers, these are not parameters and should not be changed
stale = 0
best_loss = 99999999

for epoch in range(n_epochs):

    # ---------- Training ----------
    # Make sure the model is in train mode before training.
    model.train()

    # These are used to record information in training.
    train_loss = []
#     train_accs = []

    for batch in tqdm(train_loader):

        # A batch consists of image data and corresponding labels.
        imgs, labels = batch
        #imgs = imgs.half()
        #print(imgs.shape,labels.shape)

        # Forward the data. (Make sure data and model are on the same device.)
        logits = model(imgs.to(device))

        # Calculate the cross-entropy loss.
        # We don't need to apply softmax before computing cross-entropy as it is done automatically.
        loss = criterion(logits, labels.to(device))

        # Gradients stored in the parameters in the previous step should be cleared out first.
        optimizer.zero_grad()

        # Compute the gradients for parameters.
        loss.backward()

        # Clip the gradient norms for stable training.
        grad_norm = nn.utils.clip_grad_norm_(model.parameters(), max_norm=10)

        # Update the parameters with computed gradients.
        optimizer.step()

        # Compute the accuracy for current batch.
#         acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

        # Record the loss and accuracy.
        train_loss.append(loss.item())
#         train_accs.append(acc)
        
    train_loss = sum(train_loss) / len(train_loss)
#     train_acc = sum(train_accs) / len(train_accs)

    # Print the information.
#     print(f"[ Train | {epoch + 1:03d}/{n_epochs:03d} ] loss = {train_loss:.5f}, acc = {train_acc:.5f}")
    print(f"[ Train | {epoch + 1:03d}/{n_epochs:03d} ] loss = {train_loss:.5f}")

    # ---------- Validation ----------
    # Make sure the model is in eval mode so that some modules like dropout are disabled and work normally.
    model.eval()

    # These are used to record information in validation.
    valid_loss = []
#     valid_accs = []

    # Iterate the validation set by batches.
    for batch in tqdm(valid_loader):

        # A batch consists of image data and corresponding labels.
        imgs, labels = batch
        #imgs = imgs.half()

        # We don't need gradient in validation.
        # Using torch.no_grad() accelerates the forward process.
        with torch.no_grad():
            logits = model(imgs.to(device))

        # We can still compute the loss (but not the gradient).
        loss = criterion(logits, labels.to(device))

        # Compute the accuracy for current batch.
#         acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

        # Record the loss and accuracy.
        valid_loss.append(loss.item())
#         valid_accs.append(acc)
        #break

    # The average loss and accuracy for entire validation set is the average of the recorded values.
    valid_loss = sum(valid_loss) / len(valid_loss)
#     valid_acc = sum(valid_accs) / len(valid_accs)

    # Print the information.
#     print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")


    # update logs
    if valid_loss < best_loss:
        with open(f"./{_exp_name}_log.txt","a"):
#             print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f} -> best")
            print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f} -> best")
    else:
        with open(f"./{_exp_name}_log.txt","a"):
#             print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")
            print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}")


    # save models
    if valid_loss < best_loss:
        print(f"Best model found at epoch {epoch + 1}, saving model")
        torch.save(model.state_dict(), f"{_exp_name}_best.ckpt") # only save best to prevent output memory exceed error
        best_loss = valid_loss
        stale = 0
    else:
        stale += 1
        if stale > patience:
            print(f"No improvment {patience} consecutive epochs, early stopping")
            break

cuda


  0%|          | 0/33 [00:00<?, ?it/s]

[ Train | 001/200 ] loss = 1.08004


  0%|          | 0/4 [00:00<?, ?it/s]

[ Valid | 001/200 ] loss = 1.74434 -> best
Best model found at epoch 1, saving model


...

[ Valid | 080/200 ] loss = 1.48351


  0%|          | 0/33 [00:00<?, ?it/s]

[ Train | 081/200 ] loss = 0.03351


  0%|          | 0/4 [00:00<?, ?it/s]

[ Valid | 081/200 ] loss = 1.52200
No improvment 20 consecutive epochs, early stopping


Predict test

In [12]:
# load test data
test_set = Dataset(os.path.join(_dataset_dir,"test"), tfm=test_tfm)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)

One /tmp/dataset-ntu-photo/photos/test sample /tmp/dataset-ntu-photo/photos/test/blazer/20170324131109229_500.jpg


In [13]:
# predict test data
model_best = Classifier().to(device)
model_best.load_state_dict(torch.load(f"{_exp_name}_best.ckpt"))
model_best.eval()
prediction = []
with torch.no_grad():
    for data,_ in test_loader:
        test_pred = model_best(data.to(device))
        test_label = np.argmax(test_pred.cpu().data.numpy(), axis=1)
        prediction += test_label.squeeze().tolist()

In [14]:
print(prediction)

[0, 2, 0, 0, 0, 3, 0, 0, 0, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 3, 1, 1, 1, 0, 3, 1, 1, 3, 2, 1, 1, 1, 1, 3, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 1, 3, 1, 3, 3, 3, 1, 3, 3, 1, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 3, 0, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 3, 3, 3, 3, 3, 1]


In [15]:
# get true label of test data
true_labels = []
for batch in tqdm(test_loader):

    # A batch consists of image data and corresponding labels.
    imgs, labels = batch
    true_labels += labels.squeeze().tolist()

  0%|          | 0/5 [00:00<?, ?it/s]

In [16]:
print(true_labels)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]


In [17]:
# Calculate test scores
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from mlxtend.evaluate import accuracy_score

print(classification_report(true_labels, prediction, target_names=labels_out))
matrix = confusion_matrix(true_labels, prediction)
print("Accuracy:", accuracy_score(true_labels, prediction))
print("Confusion Matrix:\n", matrix)
print("Per-Class Accuracy:")
for i in range(4):
    print(labels_out[i], "accuracy:", accuracy_score(true_labels, prediction, method='binary', pos_label=i))

              precision    recall  f1-score   support

      blazer       0.70      0.78      0.74         9
    cardigan       0.80      0.79      0.80        42
        coat       0.87      0.93      0.90        43
      jacket       0.88      0.83      0.85        52

    accuracy                           0.84       146
   macro avg       0.81      0.83      0.82       146
weighted avg       0.84      0.84      0.84       146

Accuracy: 0.8424657534246576
Confusion Matrix:
 [[ 7  0  1  1]
 [ 2 33  3  4]
 [ 0  2 40  1]
 [ 1  6  2 43]]
Per-Class Accuracy:
blazer accuracy: 0.9657534246575342
cardigan accuracy: 0.8835616438356164
coat accuracy: 0.9383561643835616
jacket accuracy: 0.8972602739726028


After testing SGD and Adam, we prefer to use Adam since it has much better accuracy rate. We guess Adam performs better than SGD because the improvement of SGD slows down in the middle and is much easier to occur early stopping since we stop the model within 20 epochs. Also by doing some reasearch we recommend lr=0.0003, weight_decay=1e-5 to be good hyperparameters.

The accuracy of blazer is the highest which we didn't expect since it has the smallest training dataset. This might happen because the testing data is to small and these data tend to fit the trained model very well. The accuracy of coat is higher than cardigan as we expected since it has larger training data. However, we didn't expect jacket to perform better than coat. This may happen due to jacket occur overfitting because of the large training dataset or because it has a larger testing set and there exist some outliers which is hard to predict.

## Q3

Import and setup

In [43]:
_exp_name = "q3"

In [41]:
# Import necessary packages.
import numpy as np
import pandas as pd
import torch
import os
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
# "ConcatDataset" and "Subset" are possibly useful when doing semi-supervised learning.
from torch.utils.data import ConcatDataset, DataLoader, Subset, Dataset
from torchvision.datasets import DatasetFolder, VisionDataset
import torchvision

# This is for the progress bar.
from tqdm.auto import tqdm
import random

In [42]:
myseed = 6666  # set a random seed for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(myseed)
torch.manual_seed(myseed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(myseed)

Transform picture

In [44]:
# Normally, We don't need augmentations in testing and validation.
# All we need here is to resize the PIL image and transform it into Tensor.
test_tfm = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# However, it is also possible to use augmentation in the testing phase.
# You may use train_tfm to produce a variety of images and then test using ensemble methods
train_tfm = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

Dataset

In [45]:
class Dataset(Dataset):

    def __init__(self,path,tfm=test_tfm,files = None):
        super(Dataset).__init__()
        self.path = path
        self.files = sorted([os.path.join(path, labels_out[i], j) for i in range(4) for j in os.listdir(os.path.join(path, labels_out[i])) if not j.startswith(".")])
        if files != None:
            self.files = files
        print(f"One {path} sample",self.files[0])
        self.transform = tfm
  
    def __len__(self):
        return len(self.files)
  
    def __getitem__(self,idx):
        fname = self.files[idx]
        im = Image.open(fname)
        im = self.transform(im)
        #im = self.data[idx]
        label = -2
        lname = fname.split("/")[-2]
        if lname == labels_out[0]:
            label = 0
        elif lname == labels_out[1]:
            label = 1
        elif lname == labels_out[2]:
            label = 2
        elif lname == labels_out[3]:
            label = 3
        else:
            label = -1
        return im,label

Training model

In [51]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.model = torchvision.models.resnet50(pretrained=True)
        # Freeze the parameters in the resnet50 pretrained model
        for param in self.model.parameters():
            param.requires_grad = False
        # modify the last layer
        self.model.fc = nn.Linear(2048, 4)

    def forward(self, x):
        out = self.model(x)
        out = out.view(out.size()[0], -1)
        return out

Execute

In [47]:
batch_size = 32
_dataset_dir = "/tmp/dataset-ntu-photo/photos"
# Construct datasets.
# The argument "loader" tells how torchvision reads the data.
train_set = Dataset(os.path.join(_dataset_dir,"train"), tfm=train_tfm)
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)
valid_set = Dataset(os.path.join(_dataset_dir,"valid"), tfm=test_tfm)
valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)

One /tmp/dataset-ntu-photo/photos/train sample /tmp/dataset-ntu-photo/photos/train/blazer/20170324101342210_500.jpg
One /tmp/dataset-ntu-photo/photos/valid sample /tmp/dataset-ntu-photo/photos/valid/blazer/20170324142214231_500.jpg


In [52]:
# "cuda" only when GPUs are available.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

# The number of training epochs and patience.
n_epochs = 200
patience = 20 # If no improvement in 'patience' epochs, early stop

# Initialize a model, and put it on the device specified.
model = Classifier().to(device)

# For the classification task, we use cross-entropy as the measurement of performance.
criterion = nn.CrossEntropyLoss()

# Initialize optimizer, you may fine-tune some hyperparameters such as learning rate on your own.
optimizer = torch.optim.Adam(model.parameters(), lr=0.0003, weight_decay=1e-5) 

# Initialize trackers, these are not parameters and should not be changed
stale = 0
best_loss = 99999999

for epoch in range(n_epochs):

    # ---------- Training ----------
    # Make sure the model is in train mode before training.
    model.train()

    # These are used to record information in training.
    train_loss = []
#     train_accs = []

    for batch in tqdm(train_loader):

        # A batch consists of image data and corresponding labels.
        imgs, labels = batch
        #imgs = imgs.half()
        #print(imgs.shape,labels.shape)

        # Forward the data. (Make sure data and model are on the same device.)
        logits = model(imgs.to(device))

        # Calculate the cross-entropy loss.
        # We don't need to apply softmax before computing cross-entropy as it is done automatically.
        loss = criterion(logits, labels.to(device))

        # Gradients stored in the parameters in the previous step should be cleared out first.
        optimizer.zero_grad()

        # Compute the gradients for parameters.
        loss.backward()

        # Clip the gradient norms for stable training.
        grad_norm = nn.utils.clip_grad_norm_(model.parameters(), max_norm=10)

        # Update the parameters with computed gradients.
        optimizer.step()

        # Compute the accuracy for current batch.
#         acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

        # Record the loss and accuracy.
        train_loss.append(loss.item())
#         train_accs.append(acc)
        
    train_loss = sum(train_loss) / len(train_loss)
#     train_acc = sum(train_accs) / len(train_accs)

    # Print the information.
#     print(f"[ Train | {epoch + 1:03d}/{n_epochs:03d} ] loss = {train_loss:.5f}, acc = {train_acc:.5f}")
    print(f"[ Train | {epoch + 1:03d}/{n_epochs:03d} ] loss = {train_loss:.5f}")

    # ---------- Validation ----------
    # Make sure the model is in eval mode so that some modules like dropout are disabled and work normally.
    model.eval()

    # These are used to record information in validation.
    valid_loss = []
#     valid_accs = []

    # Iterate the validation set by batches.
    for batch in tqdm(valid_loader):

        # A batch consists of image data and corresponding labels.
        imgs, labels = batch
        #imgs = imgs.half()

        # We don't need gradient in validation.
        # Using torch.no_grad() accelerates the forward process.
        with torch.no_grad():
            logits = model(imgs.to(device))

        # We can still compute the loss (but not the gradient).
        loss = criterion(logits, labels.to(device))

        # Compute the accuracy for current batch.
#         acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

        # Record the loss and accuracy.
        valid_loss.append(loss.item())
#         valid_accs.append(acc)
        #break

    # The average loss and accuracy for entire validation set is the average of the recorded values.
    valid_loss = sum(valid_loss) / len(valid_loss)
#     valid_acc = sum(valid_accs) / len(valid_accs)

    # Print the information.
#     print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")


    # update logs
    if valid_loss < best_loss:
        with open(f"./{_exp_name}_log.txt","a"):
#             print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f} -> best")
            print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f} -> best")
    else:
        with open(f"./{_exp_name}_log.txt","a"):
#             print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")
            print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}")


    # save models
    if valid_loss < best_loss:
        print(f"Best model found at epoch {epoch}, saving model")
        torch.save(model.state_dict(), f"{_exp_name}_best.ckpt") # only save best to prevent output memory exceed error
        best_loss = valid_loss
        stale = 0
    else:
        stale += 1
        if stale > patience:
            print(f"No improvment {patience} consecutive epochs, early stopping")
            break

cuda


  0%|          | 0/33 [00:00<?, ?it/s]

[ Train | 001/200 ] loss = 1.30227


  0%|          | 0/4 [00:00<?, ?it/s]

[ Valid | 001/200 ] loss = 1.22694 -> best
Best model found at epoch 0, saving model


...

[ Valid | 094/200 ] loss = 1.04947


  0%|          | 0/33 [00:00<?, ?it/s]

[ Train | 095/200 ] loss = 0.81988


  0%|          | 0/4 [00:00<?, ?it/s]

[ Valid | 095/200 ] loss = 0.98942
No improvment 20 consecutive epochs, early stopping


Predict test

In [53]:
# load test data
test_set = Dataset(os.path.join(_dataset_dir,"test"), tfm=test_tfm)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)

One /tmp/dataset-ntu-photo/photos/test sample /tmp/dataset-ntu-photo/photos/test/blazer/20170324131109229_500.jpg


In [54]:
# predict test data
model_best = Classifier().to(device)
model_best.load_state_dict(torch.load(f"{_exp_name}_best.ckpt"))
model_best.eval()
prediction = []
with torch.no_grad():
    for data,_ in test_loader:
        test_pred = model_best(data.to(device))
        test_label = np.argmax(test_pred.cpu().data.numpy(), axis=1)
        prediction += test_label.squeeze().tolist()

In [55]:
print(prediction)

[0, 3, 3, 0, 1, 0, 3, 0, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 3, 1, 3, 2, 3, 1, 1, 3, 1, 2, 2, 3, 1, 2, 2, 1, 1, 1, 3, 2, 1, 1, 1, 1, 1, 1, 3, 3, 3, 1, 2, 0, 2, 1, 2, 1, 2, 2, 1, 3, 1, 2, 0, 2, 2, 2, 3, 2, 2, 3, 3, 3, 2, 2, 3, 2, 2, 2, 3, 3, 1, 2, 2, 3, 1, 2, 2, 1, 2, 3, 1, 2, 2, 1, 3, 3, 3, 1, 3, 1, 3, 1, 3, 3, 1, 3, 3, 3, 3, 2, 3, 2, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 2, 2, 3, 3, 1, 1, 2, 1, 1, 3, 3, 0, 3, 1, 3, 1]


In [56]:
# get true label of test data
true_labels = []
for batch in tqdm(test_loader):

    # A batch consists of image data and corresponding labels.
    imgs, labels = batch
    true_labels += labels.squeeze().tolist()

  0%|          | 0/5 [00:00<?, ?it/s]

In [57]:
print(true_labels)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]


In [58]:
# Calculate test scores
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from mlxtend.evaluate import accuracy_score

print(classification_report(true_labels, prediction, target_names=labels_out))
matrix = confusion_matrix(true_labels, prediction)
print("Accuracy:", accuracy_score(true_labels, prediction))
print("Confusion Matrix:\n", matrix)
print("Per-Class Accuracy:")
for i in range(4):
    print(labels_out[i], "accuracy:", accuracy_score(true_labels, prediction, method='binary', pos_label=i))

              precision    recall  f1-score   support

      blazer       0.57      0.44      0.50         9
    cardigan       0.55      0.62      0.58        42
        coat       0.66      0.53      0.59        43
      jacket       0.58      0.63      0.61        52

    accuracy                           0.59       146
   macro avg       0.59      0.56      0.57       146
weighted avg       0.59      0.59      0.59       146

Accuracy: 0.589041095890411
Confusion Matrix:
 [[ 4  1  0  4]
 [ 0 26  6 10]
 [ 2  8 23 10]
 [ 1 12  6 33]]
Per-Class Accuracy:
blazer accuracy: 0.9452054794520548
cardigan accuracy: 0.7465753424657534
coat accuracy: 0.7808219178082192
jacket accuracy: 0.7054794520547946


By only training the last fully connected layer, it has worse performace compared with Q2 which trains the whole CNN network.

## Q4

In [26]:
_exp_name = "q4"

In [27]:
# Import necessary packages.
import numpy as np
import pandas as pd
import torch
import os
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
# "ConcatDataset" and "Subset" are possibly useful when doing semi-supervised learning.
from torch.utils.data import ConcatDataset, DataLoader, Subset, Dataset
from torchvision.datasets import DatasetFolder, VisionDataset
import torchvision

# This is for the progress bar.
from tqdm.auto import tqdm
import random

In [28]:
myseed = 6666  # set a random seed for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(myseed)
torch.manual_seed(myseed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(myseed)

Transform picture

In [29]:
# Normally, We don't need augmentations in testing and validation.
# All we need here is to resize the PIL image and transform it into Tensor.
test_tfm = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# However, it is also possible to use augmentation in the testing phase.
# You may use train_tfm to produce a variety of images and then test using ensemble methods
train_tfm = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

Dataset

In [30]:
class Dataset(Dataset):

    def __init__(self,path,tfm=test_tfm,files = None):
        super(Dataset).__init__()
        self.path = path
        self.files = sorted([os.path.join(path, labels_out[i], j) for i in range(4) for j in os.listdir(os.path.join(path, labels_out[i])) if not j.startswith(".")])
        if files != None:
            self.files = files
        print(f"One {path} sample",self.files[0])
        self.transform = tfm
  
    def __len__(self):
        return len(self.files)
  
    def __getitem__(self,idx):
        fname = self.files[idx]
        im = Image.open(fname)
        im = self.transform(im)
        #im = self.data[idx]
        label = -2
        lname = fname.split("/")[-2]
        if lname == labels_out[0]:
            label = 0
        elif lname == labels_out[1]:
            label = 1
        elif lname == labels_out[2]:
            label = 2
        elif lname == labels_out[3]:
            label = 3
        else:
            label = -1
        return im,label

Training model

In [31]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        model = torchvision.models.resnet50(pretrained=False)
        model.fc = nn.Linear(2048, 4)
        self.model = model

    def forward(self, x):
        out = self.model(x)
        out = out.view(out.size()[0], -1)
        return out

Execute

In [33]:
batch_size = 32
_dataset_dir = "/tmp/dataset-ntu-photo/photos"
# Construct datasets.
# The argument "loader" tells how torchvision reads the data.
train_set = Dataset(os.path.join(_dataset_dir,"train"), tfm=train_tfm)
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)
valid_set = Dataset(os.path.join(_dataset_dir,"valid"), tfm=test_tfm)
valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)

One /tmp/dataset-ntu-photo/photos/train sample /tmp/dataset-ntu-photo/photos/train/blazer/20170324101342210_500.jpg
One /tmp/dataset-ntu-photo/photos/valid sample /tmp/dataset-ntu-photo/photos/valid/blazer/20170324142214231_500.jpg


In [34]:
# "cuda" only when GPUs are available.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

# The number of training epochs and patience.
n_epochs = 200
patience = 20 # If no improvement in 'patience' epochs, early stop

# Initialize a model, and put it on the device specified.
model = Classifier().to(device)

# For the classification task, we use cross-entropy as the measurement of performance.
criterion = nn.CrossEntropyLoss()

# Initialize optimizer, you may fine-tune some hyperparameters such as learning rate on your own.
optimizer = torch.optim.Adam(model.parameters(), lr=0.0003, weight_decay=1e-5) 

# Initialize trackers, these are not parameters and should not be changed
stale = 0
best_loss = 99999999

for epoch in range(n_epochs):

    # ---------- Training ----------
    # Make sure the model is in train mode before training.
    model.train()

    # These are used to record information in training.
    train_loss = []
#     train_accs = []

    for batch in tqdm(train_loader):

        # A batch consists of image data and corresponding labels.
        imgs, labels = batch
        #imgs = imgs.half()
        #print(imgs.shape,labels.shape)

        # Forward the data. (Make sure data and model are on the same device.)
        logits = model(imgs.to(device))

        # Calculate the cross-entropy loss.
        # We don't need to apply softmax before computing cross-entropy as it is done automatically.
        loss = criterion(logits, labels.to(device))

        # Gradients stored in the parameters in the previous step should be cleared out first.
        optimizer.zero_grad()

        # Compute the gradients for parameters.
        loss.backward()

        # Clip the gradient norms for stable training.
        grad_norm = nn.utils.clip_grad_norm_(model.parameters(), max_norm=10)

        # Update the parameters with computed gradients.
        optimizer.step()

        # Compute the accuracy for current batch.
#         acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

        # Record the loss and accuracy.
        train_loss.append(loss.item())
#         train_accs.append(acc)
        
    train_loss = sum(train_loss) / len(train_loss)
#     train_acc = sum(train_accs) / len(train_accs)

    # Print the information.
#     print(f"[ Train | {epoch + 1:03d}/{n_epochs:03d} ] loss = {train_loss:.5f}, acc = {train_acc:.5f}")
    print(f"[ Train | {epoch + 1:03d}/{n_epochs:03d} ] loss = {train_loss:.5f}")

    # ---------- Validation ----------
    # Make sure the model is in eval mode so that some modules like dropout are disabled and work normally.
    model.eval()

    # These are used to record information in validation.
    valid_loss = []
#     valid_accs = []

    # Iterate the validation set by batches.
    for batch in tqdm(valid_loader):

        # A batch consists of image data and corresponding labels.
        imgs, labels = batch
        #imgs = imgs.half()

        # We don't need gradient in validation.
        # Using torch.no_grad() accelerates the forward process.
        with torch.no_grad():
            logits = model(imgs.to(device))

        # We can still compute the loss (but not the gradient).
        loss = criterion(logits, labels.to(device))

        # Compute the accuracy for current batch.
#         acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

        # Record the loss and accuracy.
        valid_loss.append(loss.item())
#         valid_accs.append(acc)
        #break

    # The average loss and accuracy for entire validation set is the average of the recorded values.
    valid_loss = sum(valid_loss) / len(valid_loss)
#     valid_acc = sum(valid_accs) / len(valid_accs)

    # Print the information.
#     print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")


    # update logs
    if valid_loss < best_loss:
        with open(f"./{_exp_name}_log.txt","a"):
#             print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f} -> best")
            print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f} -> best")
    else:
        with open(f"./{_exp_name}_log.txt","a"):
#             print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")
            print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}")


    # save models
    if valid_loss < best_loss:
        print(f"Best model found at epoch {epoch + 1}, saving model")
        torch.save(model.state_dict(), f"{_exp_name}_best.ckpt") # only save best to prevent output memory exceed error
        best_loss = valid_loss
        stale = 0
    else:
        stale += 1
        if stale > patience:
            print(f"No improvment {patience} consecutive epochs, early stopping")
            break

cuda


  0%|          | 0/33 [00:00<?, ?it/s]

[ Train | 001/200 ] loss = 1.42943


  0%|          | 0/4 [00:00<?, ?it/s]

[ Valid | 001/200 ] loss = 1.27955 -> best
Best model found at epoch 1, saving model


...

[ Valid | 028/200 ] loss = 1.26773


  0%|          | 0/33 [00:00<?, ?it/s]

[ Train | 029/200 ] loss = 1.05783


  0%|          | 0/4 [00:00<?, ?it/s]

[ Valid | 029/200 ] loss = 1.61803
No improvment 20 consecutive epochs, early stopping


Predict test

In [35]:
# load test data
test_set = Dataset(os.path.join(_dataset_dir,"test"), tfm=test_tfm)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)

One /tmp/dataset-ntu-photo/photos/test sample /tmp/dataset-ntu-photo/photos/test/blazer/20170324131109229_500.jpg


In [36]:
# predict test data
model_best = Classifier().to(device)
model_best.load_state_dict(torch.load(f"{_exp_name}_best.ckpt"))
model_best.eval()
prediction = []
with torch.no_grad():
    for data,_ in test_loader:
        test_pred = model_best(data.to(device))
        test_label = np.argmax(test_pred.cpu().data.numpy(), axis=1)
        prediction += test_label.squeeze().tolist()

In [37]:
print(prediction)

[1, 1, 1, 3, 1, 1, 1, 2, 3, 1, 1, 3, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 3, 3, 1, 1, 1, 1, 3, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 2, 1, 1, 3, 3, 1, 1, 3, 3, 2, 1, 1, 1, 1, 1, 3, 2, 1, 3, 3, 3, 3, 3, 1, 1, 1, 1, 3, 3, 1, 1, 1, 3, 1, 1, 1, 3, 1, 3, 1, 1, 1, 2, 1, 1, 1, 3, 3, 3, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 1, 3, 3, 1, 2, 3, 1, 3, 3, 1, 1, 1, 1, 3, 3, 2, 1, 3, 3, 1, 1, 3, 1, 1, 3, 3, 3, 1, 1, 1, 1]


In [38]:
# get true label of test data
true_labels = []
for batch in tqdm(test_loader):

    # A batch consists of image data and corresponding labels.
    imgs, labels = batch
    true_labels += labels.squeeze().tolist()

  0%|          | 0/5 [00:00<?, ?it/s]

In [39]:
print(true_labels)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]


In [40]:
# Calculate test scores
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from mlxtend.evaluate import accuracy_score

print(classification_report(true_labels, prediction, target_names=labels_out))
matrix = confusion_matrix(true_labels, prediction)
print("Accuracy:", accuracy_score(true_labels, prediction))
print("Confusion Matrix:\n", matrix)
print("Per-Class Accuracy:")
for i in range(4):
    print(labels_out[i], "accuracy:", accuracy_score(true_labels, prediction, method='binary', pos_label=i))

              precision    recall  f1-score   support

      blazer       0.00      0.00      0.00         9
    cardigan       0.36      0.76      0.48        42
        coat       0.43      0.07      0.12        43
      jacket       0.45      0.42      0.44        52

    accuracy                           0.39       146
   macro avg       0.31      0.31      0.26       146
weighted avg       0.39      0.39      0.33       146

Accuracy: 0.3904109589041096
Confusion Matrix:
 [[ 0  6  1  2]
 [ 0 32  1  9]
 [ 0 24  3 16]
 [ 0 28  2 22]]
Per-Class Accuracy:
blazer accuracy: 0.9383561643835616
cardigan accuracy: 0.5342465753424658
coat accuracy: 0.6986301369863014
jacket accuracy: 0.6095890410958904


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


The result with pretrained model is much better than the no pretrained one.

## Q5

The accuracy with pretrain model is far better than without pretrain model, and the model that trains all the parameters are better than the one that only trains the last fully connected network. The accuracy of each class keeps dropping from Q2 to Q4, however, the distributions are similar blazer accuracy > coat accuracy > cardigan accuracy and jacket accuracy. The accuracy of all tesing data keeps decreasing 0.8424657534246576 > 0.589041095890411 > 0.3904109589041096. We can conclude that pretraining highly influences the result and tuning all the parameters from the pretrained data is the best way to have high accuracy score.

Reference: https://colab.research.google.com/drive/15hMu9YiYjE_6HY99UXon2vKGk2KwugWu